In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [19]:
file_path = "C:/Users/User/Vscode/EDA25-1/data/naver_map_reviews_top.csv"
df = pd.read_csv(file_path)

In [20]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [21]:
categories = ['자연', '역사', '문화', '레저스포츠', '쇼핑', '음식', '숙박']
category_embeddings = model.encode(categories)

In [22]:
df['embedding'] = df['text'].apply(lambda x: model.encode(x))

In [23]:
def calc_sim(emb):
    return cosine_similarity([emb], category_embeddings)[0]

df['similarities'] = df['embedding'].apply(calc_sim)

In [ ]:
place_sims = df.groupby('place')['similarities'].apply(
    lambda x: np.mean(np.stack(x.to_list()), axis=0)
)
place_sims = pd.DataFrame(place_sims.tolist(), index=place_sims.index, columns=categories)

def top2_keywords(row):
    top_indices = np.argsort(row.values)[-2:][::-1]
    return ', '.join([categories[i] for i in top_indices])

place_sims['top_keywords'] = place_sims.apply(top2_keywords, axis=1)

save_path = "C:/Users/User/Vscode/EDA25-1/data/tourist_spot_top2_keywords.csv"
place_sims.to_csv(save_path, encoding='utf-8-sig')